In [1]:
import CropModels
from CropDataset import MyDataSet,normalize_torch,normalize_05,normalize_dataset,preprocess,preprocess_hflip,preprocess_with_augmentation
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from tensorboardX import SummaryWriter
import datetime
import os
import torch
from torch.autograd import Variable
from utils import RunningMean
import utils
from PIL import Image
import numpy as np
import random
NB_CLASS=59
SEED=888
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
torch.backends.cudnn.benchmark = True


In [2]:
BATCH_SIZE=32
IMAGE_SIZE=224    # 不同模型修改不同的Size
IMAGE_TRAIN_PRE='../data/AgriculturalDisease_trainingset/images/'
ANNOTATION_TRAIN='../data/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations_deleteNoise.json' #是否需要剔除两类异常类
IMAGE_VAL_PRE='../data/AgriculturalDisease_validationset/images/'
ANNOTATION_VAL='../data/AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations_deleteNoise.json' #是否需要剔除两类异常类
date=str(datetime.date.today())
with open(ANNOTATION_TRAIN) as datafile1:
    trainDataFram=pd.read_json(datafile1,orient='records')
with open(ANNOTATION_VAL) as datafile2: #first check if it's a valid json file or not
    validateDataFram =pd.read_json(datafile2,orient='records')    
def getmodel():
    print('[+] loading model... ', end='', flush=True)
    model=CropModels.densenet201_finetune(NB_CLASS) # 需要修改为使用的model
    model.cuda()
    print('Done')
    return model
def train(epochNum):
    writer=SummaryWriter('log/'+date+'/DesNet201/') # 创建 /log/日期/InceptionResnet的组织形式  不同模型需要修改不同名称
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    weight=torch.Tensor([1,3,3,3,3,4,2,3,3,3,3,3,3,3,3,3,2,3,3,3,2,3,4,2,3,1,1,3,2,2,1,3,3,1,3,2,3,3,3,3,2,1,3,2,3,3,3,1,3,3,4,4,3,2,2,3,1,1,3]).cuda()
    criterion=nn.CrossEntropyLoss(weight=weight).cuda()
#     lx, px = utils.predict(model,val_dataLoader)
#     min_loss = criterion(Variable(px), Variable(lx)).item()
    min_loss=4.1
    print('min_loss is :%f'%(min_loss))
    min_acc=0.80
    patience=0
    lr=0.0
    momentum=0.9
    for epoch in range(epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==3 or epoch==4 or epoch==5:
            lr=0.00003
            momentum=0.95
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if epoch==6:
            lr=1e-4
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))        
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/DesNet201/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('loss has increased lr divide 10 lr now is :%f'%(lr))
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-3
            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
#             optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)

            #optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
            #optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
            
             #optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))       
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'DesNet201', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'DesNet201', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
#train(60)

[+] loading model... 

/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Done
min_loss is :4.100000
Epoch 0/59
----------
[epoch:0,batch:29]:acc: 0.126042,loss:3.843178
[epoch:0,batch:59]:acc: 0.281250,loss:3.374204
[epoch:0,batch:89]:acc: 0.362847,loss:3.044162
[epoch:0,batch:119]:acc: 0.418490,loss:2.799988
[epoch:0,batch:149]:acc: 0.455208,loss:2.601840
[epoch:0,batch:179]:acc: 0.482465,loss:2.449963
[epoch:0,batch:209]:acc: 0.507738,loss:2.317951
[epoch:0,batch:239]:acc: 0.530208,loss:2.194325
[epoch:0,batch:269]:acc: 0.546065,loss:2.098324
[epoch:0,batch:299]:acc: 0.560104,loss:2.018067
[epoch:0,batch:299]: val_loss:1.105027,val_acc:0.722185,val_total:4539
[epoch:0,batch:329]:acc: 0.573769,loss:1.938710
[epoch:0,batch:359]:acc: 0.582031,loss:1.880606
[epoch:0,batch:389]:acc: 0.591186,loss:1.821571
[epoch:0,batch:419]:acc: 0.601265,loss:1.763964
[epoch:0,batch:449]:acc: 0.608750,loss:1.714544
[epoch:0,batch:479]:acc: 0.612891,loss:1.676296
[epoch:0,batch:509]:acc: 0.619240,loss:1.635641
[epoch:0,batch:539]:acc: 0.624421,loss:1.598293
[epoch:0,batch:569]

[epoch:4,batch:119]:acc: 0.859896,loss:0.371331
[epoch:4,batch:149]:acc: 0.861667,loss:0.363234
[epoch:4,batch:179]:acc: 0.863889,loss:0.360007
[epoch:4,batch:209]:acc: 0.863244,loss:0.360731
[epoch:4,batch:239]:acc: 0.862370,loss:0.363692
[epoch:4,batch:269]:acc: 0.861806,loss:0.363287
[epoch:4,batch:299]:acc: 0.859375,loss:0.369406
[epoch:4,batch:299]: val_loss:0.463146,val_acc:0.837850,val_total:4539
[epoch:4,batch:329]:acc: 0.858239,loss:0.371994
[epoch:4,batch:359]:acc: 0.858333,loss:0.370625
[epoch:4,batch:389]:acc: 0.856571,loss:0.372786
[epoch:4,batch:419]:acc: 0.856176,loss:0.372134
[epoch:4,batch:449]:acc: 0.853889,loss:0.376210
[epoch:4,batch:479]:acc: 0.853646,loss:0.376872
[epoch:4,batch:509]:acc: 0.853309,loss:0.377677
[epoch:4,batch:539]:acc: 0.853762,loss:0.376284
[epoch:4,batch:569]:acc: 0.854002,loss:0.378753
[epoch:4,batch:599]:acc: 0.853750,loss:0.378609
[epoch:4,batch:599]: val_loss:0.444005,val_acc:0.838731,val_total:4539
[epoch:4,batch:629]:acc: 0.853919,loss:0.3

[epoch:8,batch:179]:acc: 0.889236,loss:0.260248
[epoch:8,batch:209]:acc: 0.891220,loss:0.256016
[epoch:8,batch:239]:acc: 0.889062,loss:0.258743
[epoch:8,batch:269]:acc: 0.889236,loss:0.260323
[epoch:8,batch:299]:acc: 0.890208,loss:0.259771
[epoch:8,batch:299]: val_loss:0.419151,val_acc:0.855254,val_total:4539
[epoch:8,batch:329]:acc: 0.890152,loss:0.258388
[epoch:8,batch:359]:acc: 0.891146,loss:0.256639
[epoch:8,batch:389]:acc: 0.891346,loss:0.257104
[epoch:8,batch:419]:acc: 0.890253,loss:0.259702
[epoch:8,batch:449]:acc: 0.889931,loss:0.259357
[epoch:8,batch:479]:acc: 0.889974,loss:0.260558
[epoch:8,batch:509]:acc: 0.890012,loss:0.260686
[epoch:8,batch:539]:acc: 0.889757,loss:0.260496
[epoch:8,batch:569]:acc: 0.890022,loss:0.259752
[epoch:8,batch:599]:acc: 0.889635,loss:0.261353
[epoch:8,batch:599]: val_loss:0.422214,val_acc:0.846883,val_total:4539
[epoch:8,batch:629]:acc: 0.889534,loss:0.261662
[epoch:8,batch:659]:acc: 0.889725,loss:0.261258
[epoch:8,batch:689]:acc: 0.888678,loss:0.2

[epoch:12,batch:299]: val_loss:0.402198,val_acc:0.859440,val_total:4539
[epoch:12,batch:329]:acc: 0.907576,loss:0.219628
[epoch:12,batch:359]:acc: 0.906076,loss:0.221036
[epoch:12,batch:389]:acc: 0.906330,loss:0.220782
[epoch:12,batch:419]:acc: 0.905729,loss:0.220070
[epoch:12,batch:449]:acc: 0.905903,loss:0.220211
[epoch:12,batch:479]:acc: 0.906315,loss:0.219689
[epoch:12,batch:509]:acc: 0.906740,loss:0.218932
[epoch:12,batch:539]:acc: 0.907523,loss:0.217917
[epoch:12,batch:569]:acc: 0.907950,loss:0.216861
[epoch:12,batch:599]:acc: 0.907760,loss:0.216363
[epoch:12,batch:599]: val_loss:0.402418,val_acc:0.862965,val_total:4539
[epoch:12,batch:629]:acc: 0.907143,loss:0.218130
[epoch:12,batch:659]:acc: 0.907055,loss:0.218231
[epoch:12,batch:689]:acc: 0.907971,loss:0.217140
[epoch:12,batch:719]:acc: 0.908420,loss:0.216532
[epoch:12,batch:749]:acc: 0.908583,loss:0.216596
[epoch:12,batch:779]:acc: 0.909375,loss:0.215824
[epoch:12,batch:809]:acc: 0.909877,loss:0.215152
[epoch:12,batch:839]:ac

[epoch:16,batch:419]:acc: 0.907143,loss:0.226320
[epoch:16,batch:449]:acc: 0.906528,loss:0.226836
[epoch:16,batch:479]:acc: 0.907487,loss:0.225074
[epoch:16,batch:509]:acc: 0.906618,loss:0.225553
[epoch:16,batch:539]:acc: 0.906539,loss:0.225344
[epoch:16,batch:569]:acc: 0.906469,loss:0.224922
[epoch:16,batch:599]:acc: 0.907344,loss:0.223520
[epoch:16,batch:599]: val_loss:0.393805,val_acc:0.863406,val_total:4539
[epoch:16,batch:629]:acc: 0.907490,loss:0.223531
[epoch:16,batch:659]:acc: 0.906913,loss:0.223354
[epoch:16,batch:689]:acc: 0.907609,loss:0.222699
[epoch:16,batch:719]:acc: 0.907118,loss:0.222636
[epoch:16,batch:749]:acc: 0.907375,loss:0.222531
[epoch:16,batch:779]:acc: 0.907292,loss:0.222081
[epoch:16,batch:809]:acc: 0.907137,loss:0.222190
[epoch:16,batch:839]:acc: 0.907180,loss:0.221808
[epoch:16,batch:869]:acc: 0.906358,loss:0.222707
[epoch:16,batch:899]:acc: 0.906319,loss:0.222691
[epoch:16,batch:899]: val_loss:0.394395,val_acc:0.862745,val_total:4539
[epoch:16,batch:929]:ac

In [ ]:
def reuseTrain(path,epochNum):
    writer=SummaryWriter('log/'+date+'/DesNet201/') # 创建 /log/日期/InceptionResnet的组织形式  不同模型需要修改不同名称
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    weight=torch.Tensor([1,3,3,3,3,4,2,3,3,3,3,3,3,3,3,3,2,3,3,3,2,3,4,2,3,1,1,3,2,2,1,3,3,1,3,2,3,3,3,3,2,1,3,2,3,3,3,1,3,3,4,4,3,2,2,3,1,1,3]).cuda()
    criterion=nn.CrossEntropyLoss(weight=weight).cuda()
#     lx, px = utils.predict(model,val_dataLoader)
#     min_loss = criterion(Variable(px), Variable(lx)).item()
    min_loss=4.1
    print('min_loss is :%f'%(min_loss))
    min_acc=0.80
    patience=0
    lr=0.0
    momentum=0.0
    for epoch in range(epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==3:
            lr=1e-4
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/DesNet201/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('loss has increased lr divide 10 lr now is :%f'%(lr))
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-3
#             optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
#             optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))       
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'DesNet201', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'DesNet201', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [3]:
def TrainWithRawData(path,epochNum):
    writer=SummaryWriter('log/'+date+'/DesNet201/') # 创建 /log/日期/InceptionResnet的组织形式
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))
    print('val_correct is %f'%(modelParams['val_correct']))
    min_acc=max(modelParams['val_correct'],0.81)
    optinizerSave=modelParams['optimizer']
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==3:
            patience=0
            model.load_state_dict(torch.load('../model/DesNet201/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('lr desencd')
        if epoch==beginepoch:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer.load_state_dict(optinizerSave)
#             lr=optimizer['lr']
#             momentum=optimizer['momentum']
            print('begin lr is ',lr)
            
        else:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
                   
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
                    if  log_loss < min_loss:
                        utils.snapshot('../model/', 'DesNet201', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy })          

                        min_loss=log_loss
                        print('save new model loss,now loss is ',min_loss)

                    if accuracy>min_acc:
                        utils.snapshot('../model/', 'DesNet201', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy },key='acc') 
                        min_acc=accuracy
                        print('save new model acc,now acc is ',min_acc)
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))         
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'DesNet201', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'DesNet201', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
TrainWithRawData('../model/DesNet201/2018-11-01_acc_best.pth',60)

[+] loading model... 

/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Done
min_loss is :0.397214
val_correct is 0.861864
Epoch 11/59
----------
begin lr is  0.0001
[epoch:11,batch:29]:acc: 0.890625,loss:0.245192
[epoch:11,batch:59]:acc: 0.899479,loss:0.227566
[epoch:11,batch:89]:acc: 0.904861,loss:0.223650
[epoch:11,batch:119]:acc: 0.906510,loss:0.223769
[epoch:11,batch:149]:acc: 0.907083,loss:0.223205
[epoch:11,batch:179]:acc: 0.906597,loss:0.224726
[epoch:11,batch:209]:acc: 0.906101,loss:0.224138
[epoch:11,batch:239]:acc: 0.906901,loss:0.220967
[epoch:11,batch:269]:acc: 0.907986,loss:0.219209
[epoch:11,batch:299]:acc: 0.908542,loss:0.217059
[epoch:11,batch:299]: val_loss:0.354680,val_acc:0.866490,val_total:4539
save new model loss,now loss is  0.35468021035194397
save new model acc,now acc is  tensor(0.8665, device='cuda:0')
[epoch:11,batch:329]:acc: 0.909186,loss:0.216487
[epoch:11,batch:359]:acc: 0.909549,loss:0.215935
[epoch:11,batch:389]:acc: 0.909856,loss:0.214716
[epoch:11,batch:419]:acc: 0.910565,loss:0.213926
[epoch:11,batch:449]:acc: 0.911944,

[epoch:14,batch:929]:acc: 0.943851,loss:0.153369
[epoch:14,batch:959]:acc: 0.943848,loss:0.153175
[epoch:14,batch:989]:acc: 0.944034,loss:0.152785
[epoch:14] :acc: 0.944005,loss:0.152864,lr:0.000100,patience:2
[epoch:14]: val_loss:0.356526,val_acc:0.869134,
Epoch 15/59
----------
lr desencd
[epoch:15,batch:29]:acc: 0.940625,loss:0.158389
[epoch:15,batch:59]:acc: 0.935937,loss:0.163992
[epoch:15,batch:89]:acc: 0.937153,loss:0.161790
[epoch:15,batch:119]:acc: 0.939844,loss:0.160240
[epoch:15,batch:149]:acc: 0.942500,loss:0.158944
[epoch:15,batch:179]:acc: 0.944965,loss:0.157009
[epoch:15,batch:209]:acc: 0.944196,loss:0.158443
[epoch:15,batch:239]:acc: 0.944531,loss:0.158671
[epoch:15,batch:269]:acc: 0.944907,loss:0.158040
[epoch:15,batch:299]:acc: 0.943750,loss:0.159701
[epoch:15,batch:299]: val_loss:0.349220,val_acc:0.869795,val_total:4539
[epoch:15,batch:329]:acc: 0.943466,loss:0.160415
[epoch:15,batch:359]:acc: 0.943924,loss:0.158921
[epoch:15,batch:389]:acc: 0.942548,loss:0.160225
[e

[epoch:18]: val_loss:0.350135,val_acc:0.866711,
Epoch 19/59
----------
[epoch:19,batch:29]:acc: 0.946875,loss:0.160242
[epoch:19,batch:59]:acc: 0.945833,loss:0.154699
[epoch:19,batch:89]:acc: 0.945139,loss:0.158312
[epoch:19,batch:119]:acc: 0.942708,loss:0.160962
[epoch:19,batch:149]:acc: 0.941250,loss:0.161392
[epoch:19,batch:179]:acc: 0.940799,loss:0.161638
[epoch:19,batch:209]:acc: 0.941518,loss:0.160719
[epoch:19,batch:239]:acc: 0.940625,loss:0.161731
[epoch:19,batch:269]:acc: 0.939699,loss:0.162266
[epoch:19,batch:299]:acc: 0.940000,loss:0.161013
[epoch:19,batch:299]: val_loss:0.347885,val_acc:0.870236,val_total:4539
[epoch:19,batch:329]:acc: 0.940436,loss:0.160528
[epoch:19,batch:359]:acc: 0.940885,loss:0.159478
[epoch:19,batch:389]:acc: 0.941587,loss:0.158397
[epoch:19,batch:419]:acc: 0.940253,loss:0.160680
[epoch:19,batch:449]:acc: 0.940139,loss:0.160988
[epoch:19,batch:479]:acc: 0.939844,loss:0.162000
[epoch:19,batch:509]:acc: 0.940196,loss:0.161810
[epoch:19,batch:539]:acc: 0

[epoch:23,batch:149]:acc: 0.935000,loss:0.170019
[epoch:23,batch:179]:acc: 0.938715,loss:0.165384
[epoch:23,batch:209]:acc: 0.938393,loss:0.164417
[epoch:23,batch:239]:acc: 0.938932,loss:0.162809
[epoch:23,batch:269]:acc: 0.936921,loss:0.165972
[epoch:23,batch:299]:acc: 0.936458,loss:0.167109
[epoch:23,batch:299]: val_loss:0.349482,val_acc:0.869354,val_total:4539
[epoch:23,batch:329]:acc: 0.936932,loss:0.166577
[epoch:23,batch:359]:acc: 0.935851,loss:0.167590
[epoch:23,batch:389]:acc: 0.936058,loss:0.167469
[epoch:23,batch:419]:acc: 0.935789,loss:0.167634
[epoch:23,batch:449]:acc: 0.935972,loss:0.167277
[epoch:23,batch:479]:acc: 0.936719,loss:0.165895
[epoch:23,batch:509]:acc: 0.937255,loss:0.165263
[epoch:23,batch:539]:acc: 0.937674,loss:0.164901
[epoch:23,batch:569]:acc: 0.937884,loss:0.164390
[epoch:23,batch:599]:acc: 0.938073,loss:0.163721
[epoch:23,batch:599]: val_loss:0.350466,val_acc:0.870015,val_total:4539
[epoch:23,batch:629]:acc: 0.937897,loss:0.164197
[epoch:23,batch:659]:ac

[epoch:27,batch:329]:acc: 0.938731,loss:0.165815
[epoch:27,batch:359]:acc: 0.939236,loss:0.165240
[epoch:27,batch:389]:acc: 0.938702,loss:0.165697
[epoch:27,batch:419]:acc: 0.938616,loss:0.165371
[epoch:27,batch:449]:acc: 0.938681,loss:0.165224
[epoch:27,batch:479]:acc: 0.938281,loss:0.164935
[epoch:27,batch:509]:acc: 0.937990,loss:0.165564
[epoch:27,batch:539]:acc: 0.938831,loss:0.164843
[epoch:27,batch:569]:acc: 0.938871,loss:0.164867
[epoch:27,batch:599]:acc: 0.939688,loss:0.163945
[epoch:27,batch:599]: val_loss:0.348760,val_acc:0.870236,val_total:4539
[epoch:27,batch:629]:acc: 0.939484,loss:0.163792
[epoch:27,batch:659]:acc: 0.939110,loss:0.163916
[epoch:27,batch:689]:acc: 0.939176,loss:0.163289
[epoch:27,batch:719]:acc: 0.938585,loss:0.163906
[epoch:27,batch:749]:acc: 0.938958,loss:0.163441
[epoch:27,batch:779]:acc: 0.939143,loss:0.163278
[epoch:27,batch:809]:acc: 0.939429,loss:0.162837
[epoch:27,batch:839]:acc: 0.939025,loss:0.163202
[epoch:27,batch:869]:acc: 0.938757,loss:0.1636

In [5]:
modelParams=torch.load('../model/DesNet201/2018-11-01_acc_best.pth')
min_loss=modelParams['val_loss']
print('min_loss is :%f'%(min_loss))
print('val_correct is %f'%(modelParams['val_correct']))

min_loss is :0.349666
val_correct is 0.871778
